In [1]:
import pandas as pd
import os
import re
records = pd.read_excel(r"Result/records1.xlsx", header=[0], index_col = [0])
records

,Problem(file name),#vars,#assertions,yices2_10_Result,yices2_10_Timeout,yices2_1200_Result,yices2_1200_Timeout,yices2_2400_Result,yices2_2400_Timeout,yices2_300_Result,...,mathsat_1200_Result,mathsat_1200_Timeout,mathsat_2400_Result,mathsat_2400_Timeout,mathsat_300_Result,mathsat_300_Timeout,mathsat_60_Result,mathsat_60_Timeout,mathsat_600_Result,mathsat_600_Timeout
0,NaN,54,6,sat\n,538 ms,sat\n,671 ms,sat\n,982 ms,sat\n,...,sat\n,231 ms,sat\n,223 ms,sat\n,207 ms,sat\n,200 ms,sat\n,215 ms
1,NaN,40,5,unknown\n,10031 ms,sat\n,18694 ms,sat\n,26310 ms,sat\n,...,sat\n,817 ms,sat\n,791 ms,sat\n,768 ms,sat\n,588 ms,sat\n,616 ms
2,NaN,57,6,sat\n,851 ms,sat\n,1568 ms,sat\n,1550 ms,sat\n,...,sat\n,9766 ms,sat\n,9397 ms,sat\n,8828 ms,sat\n,7881 ms,sat\n,8678 ms
3,NaN,30,4,unsat\n,962 ms,unsat\n,1630 ms,unsat\n,1870 ms,unsat\n,...,unsat\n,489 ms,unsat\n,445 ms,unsat\n,454 ms,unsat\n,464 ms,unsat\n,434 ms
4,NaN,47,5,unknown\n,10032 ms,unknown\n,1200315 ms,sat\n,2359796 ms,unknown\n,...,sat\n,551606 ms,sat\n,521194 ms,timeout,300445 ms,timeout,60227 ms,sat\n,478341 ms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24253,NaN,47,1,sat\n,29 ms,sat\n,26 ms,sat\n,34 ms,sat\n,...,sat\n,53 ms,sat\n,62 ms,sat\n,66 ms,sat\n,67 ms,sat\n,66 ms
24254,NaN,123,1,unsat\n,150 ms,unsat\n,227 ms,unsat\n,242 ms,unsat\n,...,unsat\n,470 ms,unsat\n,524 ms,unsat\n,527 ms,unsat\n,528 ms,unsat\n,477 ms
24255,NaN,162,1,unsat\n,430 ms,unsat\n,534 ms,unsat\n,825 ms,unsat\n,...,unsat\n,1536 ms,unsat\n,1625 ms,unsat\n,1515 ms,unsat\n,1420 ms,unsat\n,1143 ms
24256,NaN,41,1,sat\n,23 ms,sat\n,67 ms,sat\n,35 ms,sat\n,...,sat\n,42 ms,sat\n,43 ms,sat\n,45 ms,sat\n,51 ms,sat\n,29 ms


保存smt的变量、限制等信息

In [ ]:
Secondary_file = ['20170427-VeryMax', '2019-ezsmt', '20210219-Dartagnan']
for sf in Secondary_file:
    path = "../QF_NIA/"+sf
    folder_list = os.listdir(path)
    for filefolder in folder_list:
        folder_path = path + "/" + filefolder
        file_list = os.listdir(folder_path)
        total = len(file_list)
        count = 1
        for filename in file_list:
            with open(os.path.join(folder_path, filename)) as f:
                vars = 0
                assertions = 0
                line = f.readline()
                while line:
                    if line[:3] == '(de':
                        vars += 1
                    elif line[:3] == '(as':
                        assertions += 1
                    line = f.readline()
                print(count, "/", total,": ", filename, "+", vars, "+", assertions)
                count += 1
                records = records.append({'filename': sf+'/' + filefolder + '/' + filename, '#vars':vars, '#assertions':assertions}, ignore_index=True, verify_integrity=True)

path = "../QF_NIA"
folder_list = os.listdir(path)
folder_list.remove('2019-ezsmt')
folder_list.remove('20170427-VeryMax')
folder_list.remove('20210219-Dartagnan')
for filefolder in folder_list:
    folder_path = path + "/" + filefolder
    file_list = os.listdir(folder_path)
    total = len(file_list)
    count = 1
    for filename in file_list:
        with open(os.path.join(folder_path, filename)) as f:
            vars = 0
            assertions = 0
            line = f.readline()
            while line:
                if line[:3] == '(de':
                    vars += 1
                elif line[:3] == '(as':
                    assertions += 1
                line = f.readline()
            print(filefolder, "(",count, "/", total,"): ", filename, "+", vars, "+", assertions)
            count += 1
            records = records.append({'filename': filefolder + '/' + filename, '#vars':vars, '#assertions':assertions}, ignore_index=True, verify_integrity=True)
print("Waiting...")
records.to_excel('Result/records.xlsx')
print("Done")

修改filename未测试路径, 保存为z3的索引

In [10]:
folder_path = "Data_File/z3/10"
file_list = os.listdir(folder_path)
total = len(file_list)
count = 1
for filename in file_list:
    with open(os.path.join(folder_path, filename)) as f:
        while f.readline():
            result = f.readline()
            info = f.readline().split(' : ')
            records.loc[records['filename'] == info[0].split('//')[-1], 'Problem(file name)'] = info[0]
    print("(",count, "/", total,"): ", filename)
    count += 1
print("Waiting...")
records.to_excel('Result/records.xlsx')
print("Done")


( 1 / 49 ):  qfnia_1.log
( 2 / 49 ):  qfnia_10.log
( 3 / 49 ):  qfnia_11.log
( 4 / 49 ):  qfnia_2_1.log
( 5 / 49 ):  qfnia_2_10.log
( 6 / 49 ):  qfnia_2_11.log
( 7 / 49 ):  qfnia_2_12.log
( 8 / 49 ):  qfnia_2_13.log
( 9 / 49 ):  qfnia_2_14.log
( 10 / 49 ):  qfnia_2_15.log
( 11 / 49 ):  qfnia_2_16.log
( 12 / 49 ):  qfnia_2_17.log
( 13 / 49 ):  qfnia_2_18.log
( 14 / 49 ):  qfnia_2_19.log
( 15 / 49 ):  qfnia_2_2.log
( 16 / 49 ):  qfnia_2_20.log
( 17 / 49 ):  qfnia_2_21.log
( 18 / 49 ):  qfnia_2_22.log
( 19 / 49 ):  qfnia_2_23.log
( 20 / 49 ):  qfnia_2_24.log
( 21 / 49 ):  qfnia_2_25.log
( 22 / 49 ):  qfnia_2_26.log
( 23 / 49 ):  qfnia_2_27.log
( 24 / 49 ):  qfnia_2_28.log
( 25 / 49 ):  qfnia_2_29.log
( 26 / 49 ):  qfnia_2_3.log
( 27 / 49 ):  qfnia_2_30.log
( 28 / 49 ):  qfnia_2_31.log
( 29 / 49 ):  qfnia_2_32.log
( 30 / 49 ):  qfnia_2_33.log
( 31 / 49 ):  qfnia_2_34.log
( 32 / 49 ):  qfnia_2_35.log
( 33 / 49 ):  qfnia_2_36.log
( 34 / 49 ):  qfnia_2_4.log
( 35 / 49 ):  qfnia_2_5.log
( 36 /

保存求解器得到的result、timeout等信息

In [2]:
path = "Data_File"
folder_list = os.listdir(path)
for filefolder in folder_list:
    if filefolder != 'smtrat' and filefolder != 'ismt':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        else:
                            if result[:2] == './' or result[:2] == 'cv' or result[:2] == 'ca':
                                result = 'timeout'
                            elif result[:2] == 'tc':
                                result = 'leak'
                            info = f.readline()
                            if info[:2] == '(e':
                                break
                            while info[:2] == './' or info[:2] == 'tc' or info[:2] == 'cv' or info[:2] == 'ca':
                                info = f.readline()
                            if info[:2] != '..':
                                result = info
                                info = f.readline()
                        info = info.split(' : ')
                        records.loc[records['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                count += 1
 

Data_File/cvc5/10 ( 1 / 49 ):  qfnia_1.log
Data_File/cvc5/10 ( 2 / 49 ):  qfnia_10.log
Data_File/cvc5/10 ( 3 / 49 ):  qfnia_11.log
Data_File/cvc5/10 ( 4 / 49 ):  qfnia_2_1.log
Data_File/cvc5/10 ( 5 / 49 ):  qfnia_2_10.log
Data_File/cvc5/10 ( 6 / 49 ):  qfnia_2_11.log
Data_File/cvc5/10 ( 7 / 49 ):  qfnia_2_12.log
Data_File/cvc5/10 ( 8 / 49 ):  qfnia_2_13.log
Data_File/cvc5/10 ( 9 / 49 ):  qfnia_2_14.log
Data_File/cvc5/10 ( 10 / 49 ):  qfnia_2_15.log
Data_File/cvc5/10 ( 11 / 49 ):  qfnia_2_16.log
Data_File/cvc5/10 ( 12 / 49 ):  qfnia_2_17.log
Data_File/cvc5/10 ( 13 / 49 ):  qfnia_2_18.log
Data_File/cvc5/10 ( 14 / 49 ):  qfnia_2_19.log
Data_File/cvc5/10 ( 15 / 49 ):  qfnia_2_2.log
Data_File/cvc5/10 ( 16 / 49 ):  qfnia_2_20.log
Data_File/cvc5/10 ( 17 / 49 ):  qfnia_2_21.log
Data_File/cvc5/10 ( 18 / 49 ):  qfnia_2_22.log
Data_File/cvc5/10 ( 19 / 49 ):  qfnia_2_23.log
Data_File/cvc5/10 ( 20 / 49 ):  qfnia_2_24.log
Data_File/cvc5/10 ( 21 / 49 ):  qfnia_2_25.log
Data_File/cvc5/10 ( 22 / 49 ): 

In [ ]:
path = "Data_File"
folder_list = os.listdir(path)
for filefolder in folder_list:
    if filefolder == 'smtrat':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        else:
                            if result[:2] == './' or result[:2] == 'cv' or result[:2] == 'ca':
                                result = 'timeout'
                            elif result[:2] == 'tc':
                                result = 'leak'
                            info = f.readline()
                            if info[:2] == '(e':
                                break
                            while info[:2] == './' or info[:2] == 'tc' or info[:2] == 'cv' or info[:2] == 'ca':
                                info = f.readline()
                            if info[:2] != '..':
                                result = info
                                info = f.readline()
                        info = info.split(' : ')
                        records.loc[records['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                count += 1

In [3]:
print("Waiting...")
records.to_excel('Result/records1.xlsx')
print("Done")

Waiting...
Done


保存ismt的结果和时间

In [2]:
records1 = pd.read_excel(r"Result/records1.xlsx", header=[0], index_col = [0])
records2 = pd.read_excel(r"Result/records1.xlsx", header=[0], index_col = [0])
path = "Data_File"
folder_list = os.listdir(path)
for filefolder in folder_list:
    if filefolder == 'ismt':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        else:
                            info = f.readline()
                            while info[:2] != '..':
                                result = info
                                info = f.readline()
                            if result[:2] == 're':
                                result = 'sat'
                            elif result[:2] == 'Nu':
                                result = 'timeout'
                        info = info.split(' : ')
                        records1.loc[records1['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                        records2.loc[records2['filename'] == info[0].split('//')[1], ['Problem(file name)', result_index, timeout_index, 'group']] = [info[0], result, info[1][:-2], 1]
                count += 1

Data_File/ismt/10 ( 1 / 43 ):  qfnia_1.log
Data_File/ismt/10 ( 2 / 43 ):  qfnia_10.log
Data_File/ismt/10 ( 3 / 43 ):  qfnia_11.log
Data_File/ismt/10 ( 4 / 43 ):  qfnia_2_1.log
Data_File/ismt/10 ( 5 / 43 ):  qfnia_2_10.log
Data_File/ismt/10 ( 6 / 43 ):  qfnia_2_11.log
Data_File/ismt/10 ( 7 / 43 ):  qfnia_2_12.log
Data_File/ismt/10 ( 8 / 43 ):  qfnia_2_13.log
Data_File/ismt/10 ( 9 / 43 ):  qfnia_2_14.log
Data_File/ismt/10 ( 10 / 43 ):  qfnia_2_15.log
Data_File/ismt/10 ( 11 / 43 ):  qfnia_2_16.log
Data_File/ismt/10 ( 12 / 43 ):  qfnia_2_17.log
Data_File/ismt/10 ( 13 / 43 ):  qfnia_2_18.log
Data_File/ismt/10 ( 14 / 43 ):  qfnia_2_19.log
Data_File/ismt/10 ( 15 / 43 ):  qfnia_2_2.log
Data_File/ismt/10 ( 16 / 43 ):  qfnia_2_20.log
Data_File/ismt/10 ( 17 / 43 ):  qfnia_2_21.log
Data_File/ismt/10 ( 18 / 43 ):  qfnia_2_22.log
Data_File/ismt/10 ( 19 / 43 ):  qfnia_2_23.log
Data_File/ismt/10 ( 20 / 43 ):  qfnia_2_24.log
Data_File/ismt/10 ( 21 / 43 ):  qfnia_2_25.log
Data_File/ismt/10 ( 22 / 43 ): 

In [3]:
records1 = records1.drop(columns = ['filename'])
records2 = records2.drop(columns = ['filename'])
records2 = records2[records2['group'] == 1]
records2 = records2.drop(columns = ['group'])
print("Waiting...")
records1.to_excel('Result/result1.xlsx')
records2.to_excel('Result/result2.xlsx')
print("Done")

Waiting...
Done
